In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score

In [2]:
def create_layer(n_in, n_out, act_func = None):
    layer = {}
    
    layer["weights"] = np.random.random((n_out, n_in + 1))
    layer["act_func"] = act_func()
    
    return layer

In [3]:
def pass_layer(layer, X):
    X = np.concatenate((X, np.ones((X.shape[0], 1))), axis = 1)

    temp = torch.tensor((X @ layer["weights"].T).astype(np.float32))
    return layer["act_func"](temp).numpy()

In [4]:
def create_network(layers, act_funcs):
    network = {}
    network["layers"] = []
    network["layer_sizes"] = []

    for i in range(len(layers) - 1):
        network["layers"] += [create_layer(layers[i], layers[i + 1], act_funcs[i])]
        network["layer_sizes"] += [(layers[i] + 1, layers[i + 1])]

    return network

In [5]:
def pass_network(network, X):
    for layer in network["layers"]:
        X = pass_layer(layer, X)
    return X

In [6]:
def get_weight_vector(network):
    weight_vector = []
    
    for layer in network["layers"]:
        weight_vector += [layer["weights"].flatten()]
        
    weight_vector = np.concatenate(weight_vector)
    return weight_vector

In [7]:
def set_layer_weights(network, weight_vector):
    start = 0
    
    for l in range(len(network["layer_sizes"])):
        input_size, output_size = network["layer_sizes"][l]
        weights = weight_vector[start:start + input_size*output_size]
        weights = weights.reshape((output_size, input_size))
        network["layers"][l]["weights"] = weights
        start += input_size * output_size

In [8]:
def create_aco(model, X, Y):
    aco = {}
    aco["model"] = model
    aco["X"] = X
    aco["Y"] = Y
    aco["ind_size"] = get_weight_vector(model).shape[0]
    return aco

In [9]:
def first_batch(aco, pop_size):
    aco["pop_size"] = pop_size
    means, stds = np.random.randint(100,  size = pop_size), np.random.randint(100,  size = pop_size)
    
    aco["ant"] = []
    
    for i in range(pop_size):
        aco["ant"] += [means[i] + stds[i] * np.random.randn(aco["ind_size"])]
    aco["ant"] = np.array(aco["ant"])

In [10]:
def evaluate(aco):
    scores = []
    for ant in aco["ant"]:
        set_layer_weights(aco["model"], ant)
        pred = np.round(pass_network(aco["model"], aco["X"]))
        scores += [accuracy_score(aco["Y"], pred)]
    
    return np.array(scores)

In [11]:
def pher_calc(aco):
    scores = evaluate(aco)
    aco["pher"] = scores / sum(scores)
    aco["std"] = 1 / scores

In [12]:
def select(prob):
    total = np.cumsum(prob)
    choice = np.random.random()
    
    return np.where(total >= choice)[0][0]

In [13]:
def next_batch(aco):
    next_batch = []
    
    for i in range(aco["pop_size"]):
        next_batch += [[0 for i in range(aco["ind_size"])]]
        
        for j in range(aco["ind_size"]):
            next_ant = select(aco["pher"])
            
            next_batch[i][j] = aco["ant"][next_ant][j] + aco["std"][next_ant] * np.random.randn()
            
    next_batch = np.array(next_batch)
    aco["ant"] = next_batch.copy()

In [14]:
data = pd.read_csv("Bank_Personal_Loan_Modelling.csv")
data = data.drop(["ID", "ZIP Code"], axis = 1)

In [15]:
X = data.drop("Personal Loan", axis = 1)
y = data["Personal Loan"].values
y = y.reshape(-1, 1)

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [17]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [18]:
model = create_network([11, 4, 1], [nn.Sigmoid, nn.Sigmoid])
pass_network(model, X_train)

array([[0.9223066 ],
       [0.9403717 ],
       [0.8396028 ],
       ...,
       [0.77922946],
       [0.87868464],
       [0.8794223 ]], dtype=float32)

In [19]:
epochs = 100
aco = create_aco(model, X_train, y_train)

first_batch(aco, 100)

In [20]:
for epoch in range(epochs):
    pher_calc(aco)
    next_batch(aco)
    
    l = max(evaluate(aco))
    print(f"epoch : {epoch}  :  {l}")

epoch : 0  :  0.904
epoch : 1  :  0.904
epoch : 2  :  0.904
epoch : 3  :  0.904
epoch : 4  :  0.904
epoch : 5  :  0.904
epoch : 6  :  0.904
epoch : 7  :  0.904
epoch : 8  :  0.904
epoch : 9  :  0.904
epoch : 10  :  0.904
epoch : 11  :  0.904
epoch : 12  :  0.904
epoch : 13  :  0.904
epoch : 14  :  0.904
epoch : 15  :  0.904
epoch : 16  :  0.904
epoch : 17  :  0.904
epoch : 18  :  0.904
epoch : 19  :  0.904
epoch : 20  :  0.904
epoch : 21  :  0.904
epoch : 22  :  0.904
epoch : 23  :  0.904
epoch : 24  :  0.904
epoch : 25  :  0.904
epoch : 26  :  0.904
epoch : 27  :  0.904
epoch : 28  :  0.904
epoch : 29  :  0.904
epoch : 30  :  0.904
epoch : 31  :  0.904
epoch : 32  :  0.904
epoch : 33  :  0.904
epoch : 34  :  0.904
epoch : 35  :  0.904
epoch : 36  :  0.904
epoch : 37  :  0.904
epoch : 38  :  0.904
epoch : 39  :  0.904
epoch : 40  :  0.904
epoch : 41  :  0.90425
epoch : 42  :  0.904
epoch : 43  :  0.904
epoch : 44  :  0.904
epoch : 45  :  0.904
epoch : 46  :  0.904
epoch : 47  :  0.904
